参考

- [https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/](https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/)
- [https://python.langchain.com/v0.2/docs/integrations/providers/pinecone/](https://python.langchain.com/v0.2/docs/integrations/providers/pinecone/)
- [https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)
- [https://api.python.langchain.com/en/latest/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html](https://api.python.langchain.com/en/latest/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)
- [https://app.pinecone.io/](https://app.pinecone.io/)

In [1]:
# BAAI
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import numpy as np

load_dotenv()
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
text_li = ['How is the weather today?', '今天天气怎么样?']
doc_result = embedding.embed_documents(text_li)
print(f"doc_result[0]的维度: {len(doc_result[0])}")
print(f"doc_result[1]的维度: {len(doc_result[1])}")
print(f"doc_result[0]的模: {np.linalg.norm(doc_result[0])}")
print(f"doc_result[1]的模: {np.linalg.norm(doc_result[1])}")
cosine_similarity_normalized = np.dot(doc_result[0], doc_result[1])
print(f"doc_result[0]与doc_result[1]之间的相似度: {cosine_similarity_normalized}")

/Users/feiyu/miniconda3/envs/langchain-py11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/feiyu/miniconda3/envs/langchain-py11/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/feiyu/miniconda3/envs/langchain-py11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `r

doc_result[0]的维度: 1024
doc_result[1]的维度: 1024
doc_result[0]的模: 0.9999999668938295
doc_result[1]的模: 1.0000000349748062
doc_result[0]与doc_result[1]之间的相似度: 0.6709808574437868


In [2]:
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
import time

load_dotenv()
pc = Pinecone()
index_name = "horror-demo"  # change if desired
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [3]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.utilities import SQLDatabase
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
# Initialize the database
db = SQLDatabase.from_uri("sqlite:///db/db/horror.db")

# Fetch the content
result = db.run("SELECT content FROM wangyi LIMIT 100;", fetch="cursor")

# Split content into multiple documents if the length exceeds 100 characters
docs = []
for doc in result.mappings():
    content = doc['content']
    while len(content) > 100:
        split_content = content[:100]
        docs.append(split_content)
        content = content[100:]
    if content:
        docs.append(content)

# Initialize the Pinecone vector store
docsearch = PineconeVectorStore(embedding=embedding, index_name=index_name)

# Define the batch size
batch_size = 10
total_batches = len(docs) // batch_size + (1 if len(docs) % batch_size != 0 else 0)

# Add documents in batches with a progress bar showing percentage
with tqdm(total=total_batches, desc="Progress", unit="batch", ncols=100) as pbar:
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i + batch_size]
        docsearch.add_texts(batch_docs)
        pbar.update(1)


Progress: 100%|████████████████████████████████████████████████| 197/197 [21:46<00:00,  6.63s/batch]


In [6]:
query = "卡车司机的鬼故事"
docs = docsearch.similarity_search(query)
for doc in docs:
    print(doc)

page_content='车里伸出头嚷道：“前车加完没，还能动弹不，小伙儿，你见到鬼了，加完你倒是让他开走啊。”听到后车不友好的吵嚷，小涛非但不生气反倒高兴起来，心想：太好了，不是鬼车。这辆神秘的车也马上开走了。这一晚平安无事'
page_content='伙人车控透过舷凑到一块，摆摆龙门阵谈论鬼神，也就当做消遣了。要说这里头最吸引人的，那该算是那些鬼啊神啊之类的段子了。谁谁谁喝喜酒喝醉了，走夜路碰到鬼，倒在坟地里，睡了一晚上。谁谁谁去河里洗衣服，看见河'
page_content='5万简介：（灵车：运载灵柩或骨灰盒的车辆，你也可以理解为死人专用车。）我做了四年公交司机，心中的秘密也整整压抑了四年，我来亲身讲述你所不知道的列车惊悚事件。灵车改装成公交车之事，或许你没经历过，但你所'
page_content='渔夫这才放下了悬着的心.......自此，这些所谓辟邪打招呼的方法也被广泛的应用起来，不光是因为怕撞到不干净的东西，更是让人们放下心理作用的一个方式。故事二：二中坡的酒后见鬼经历有一个在深圳湖贝路贩卖'


In [9]:
retriever = docsearch.as_retriever(search_type="mmr")
query = "卡车司机的鬼故事"
matched_docs = retriever.invoke(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

呀

## Document 1

纸条上面写着：我们都很喜欢听你讲鬼故事……

## Document 2

、包容的姿态，欢迎全球各地的朋友前来观光旅游、投资兴业！

## Document 3

么随便的，没有休息好，就请假回去睡一觉。婷姐却不以为意，说伟哥平常都非常好说话，只要没什么活干，一般说身体不舒服，基本都批假，让我们休息时间，甚至外出，都不会管我们。“我刚刚见你没有去大保健，对你的印


### MMR检索

MMR (Maximal Marginal Relevance) 是一种用于提高检索结果多样性的算法。它的主要特点和工作原理如下:

1. 目的:在保证结果相关性的同时,减少检索结果的冗余,提高多样性[1][2].

2. 核心思想:通过贪心策略选择结果,在相关性和多样性之间寻求平衡[4].

3. 算法流程:
   - 首先选择与查询最相关的文档
   - 然后迭代选择后续文档,每次选择时考虑两个因素:
     a) 与查询的相关度
     b) 与已选文档的差异度[2][4]

4. 数学表达:
   MMR = argmax[λ * Sim1(Di, Q) - (1-λ) * max Sim2(Di, Dj)]
   其中:
   - Sim1衡量文档与查询的相似度
   - Sim2衡量文档间的相似度
   - λ是平衡相关性和多样性的参数[4]

5. 应用:广泛应用于搜索引擎、推荐系统等领域,用于结果重排序[4][5].

6. 优点:
   - 实现简单,计算效率高
   - 可以有效提高结果多样性
   - 参数λ可调,灵活控制相关性与多样性的权重[2][4]

7. 扩展:可以根据具体应用场景,选择不同的相似度计算方法,如基于内容特征、用户行为等[4].

MMR算法通过在相关性和多样性之间寻求平衡,能够有效提升检索结果的质量,为用户提供更全面、丰富的信息。

Citations:
- [1] https://ask.zkbhj.com
- [2] https://blog.csdn.net/qq_39388410/article/details/109706683
- [3] https://blog.csdn.net/sjyttkl/article/details/106896936
- [4] https://juejin.cn/post/7292628510844223523
- [5] https://www.youtube.com/watch?v=tCa4yackga0

In [10]:
query = "卡车司机的鬼故事"
found_docs = docsearch.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. 车里伸出头嚷道：“前车加完没，还能动弹不，小伙儿，你见到鬼了，加完你倒是让他开走啊。”听到后车不友好的吵嚷，小涛非但不生气反倒高兴起来，心想：太好了，不是鬼车。这辆神秘的车也马上开走了。这一晚平安无事 

2. 5万简介：（灵车：运载灵柩或骨灰盒的车辆，你也可以理解为死人专用车。）我做了四年公交司机，心中的秘密也整整压抑了四年，我来亲身讲述你所不知道的列车惊悚事件。灵车改装成公交车之事，或许你没经历过，但你所 



### `k` 和 `fetch_k` 两个参数的含义:

在 `max_marginal_relevance_search` 函数中,`k` 和 `fetch_k` 这两个参数在最大边际相关性(MMR)搜索算法中扮演不同的角色:

1. `k`: 这个参数决定了最终返回的文档数量。它代表了MMR算法最终输出的多样化且相关的结果数量。

2. `fetch_k`: 这个参数指定了在应用MMR算法之前初始检索的文档数量。它决定了从向量存储中最初获取多少文档来考虑多样性和相关性。

MMR算法的工作流程如下:

1. 首先检索与查询最相似的 `fetch_k` 个文档。
2. 然后从这 `fetch_k` 个文档中,使用MMR标准选择 `k` 个文档,这个标准在查询相关性和所选文档的多样性之间取得平衡。

通过将 `fetch_k` 设置得比 `k` 大,你允许算法考虑更大范围的潜在相关文档,这可能会导致最终结果更加多样化。当你想增加返回文档的多样性,同时又要保持与查询的相关性时,这种方法特别有用。

例如,如果你设置 `k=2` 和 `fetch_k=10`,该函数将:

1. 最初检索与查询最相似的10个文档。
2. 从这10个文档中,根据MMR标准选择并返回最能平衡相关性和多样性的2个文档。

这种方法允许在结果多样性和计算成本之间进行权衡,因为增加 `fetch_k` 会提供更多的多样化选择选项,但也需要更多的处理时间。